In [1]:
import networkx as nx
from collections import defaultdict
from csv import DictReader
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import normalized_mutual_info_score, homogeneity_score
import random
import time
from igraph import *
import numpy as np

### import graph data

In [3]:
df = pd.read_csv("data/str_edges_2011_2_2013_1.csv")
df["sum"] = df[["h0", "h1", "h2", "h3", "h4", "h5", "h6"]].sum(axis=1)
df.head()

node1                               node2  h0  \
0  1111219E1WdFoSBE3WVmY3uvHkq2NvnuV  1969VR2qCchXMW94tpcYirbVLUfFw4Pw7b   0   
1  111133WxWTrHrru31e3cBPWGErh7Vrifr  115SszjCNfn8daZxfnN4C9EW4fBLknXPKX   0   
2  111133WxWTrHrru31e3cBPWGErh7Vrifr  14j7F9Sj75zaRhEtBQb5jVyn7VgvucP3Yw   0   
3  111133WxWTrHrru31e3cBPWGErh7Vrifr  14WChGtct4AgzUZea7YT583DX4R6MfHHVB   0   
4  111133WxWTrHrru31e3cBPWGErh7Vrifr  172oaECE1gbHWevs8HCqfAMC1Ru3YnmW1T   0   

   h1  h2  h3  h4  h5  h6  sum  
0   2   0   2   2   0   2    8  
1   1   0   1   1   0   1    4  
2   1   0   1   1   0   1    4  
3   1   0   1   1   0   1    4  
4   1   0   1   1   0   1    4

### import ground truth

In [5]:
df_gt = pd.read_csv('data/ground_truth_id.csv.zip')
print(df_gt.shape)

df_gt = df_gt.dropna(axis=0, how='any', subset=["address", "entity"])
print(df_gt.shape)

df_gt = df_gt.drop_duplicates(subset="address", keep=False)
print(df_gt.shape)

entity_counts = df_gt["entity"].value_counts()
rare_entities = entity_counts[entity_counts < 10].index
df_gt = df_gt.loc[~df_gt["entity"].isin(rare_entities), :]
print(df_gt.shape)

df_gt.head()

(28027931, 4)
(28027931, 4)
(28027089, 4)
(28027060, 4)


address       entity    sector          id
0  1E421ChpHWEqQ2Kcy2JqEE8eDx7nFN4AXj  777Coin.com  Gambling  34916770.0
1  15uWGv4QFoa7Hjkfzs6YCDVGNhxZ4Yz2wW  777Coin.com  Gambling  26152409.0
2  19s8Jzb6fQ9kLu3Gcb46aX5h9uc93PULgC  777Coin.com  Gambling  29354407.0
3  1LfiL43VHwe2tRNjra5uhVAc4J5nFUay5X  777Coin.com  Gambling  29902318.0
4  1Bh6y3BkJVzodfGfLh9GCuuKPqgwoyRibF  777Coin.com  Gambling  29739794.0

In [6]:
gt_addr = set(df_gt["address"])
sample_addr = set(df["node1"]).union(set(df["node2"]))
known_addr = sample_addr.intersection(gt_addr)

print('ground truth addresses',len(gt_addr))
print('sample addressses:',len(sample_addr))
print('ground truth addresses in sample',len(known_addr))

ground truth addresses 28027060
sample addressses: 139906
ground truth addresses in sample 45603


In [7]:
df_gt_known = df_gt.loc[df_gt["address"].isin(sample_addr), ['address', 'entity']]

known_entities = set(df_gt_known['entity'])

# known_entity_addr_dict = {idx: set(df_gt_known.loc[df_gt_known["entity"] == e, "address"]) for idx, e in enumerate(known_entities)}

known_addr_entity_dict_list = [{a: idx for a in set(df_gt_known.loc[df_gt_known["entity"] == e, "address"])} for idx, e in enumerate(known_entities)]
known_addr_entity_dict = {}
[known_addr_entity_dict.update(d) for d in known_addr_entity_dict_list]

known_entity_counts = df_gt_known['entity'].value_counts().rename("count").to_frame()

print('ground truth entities in sample:',len(known_entities))
known_entity_counts

ground truth entities in sample: 42


count
Instawallet.org        18279
CoinJoinMess           11119
SilkRoadMarketplace     9678
BTC-e.com               3923
Bitcoin.de               756
BitBargain.co.uk         356
BTCJam.com               305
BitZino.com              191
Bitstamp.net             187
Bitcoin-24.com           115
SealsWithClubs.eu         87
BTCC.com                  80
Bitcash.cz                76
LocalBitcoins.com         76
BitcoinFog                43
Cavirtex.com              38
BitPay.com                36
CampBX.com                35
VirWoX.com                33
Bitmit.net                32
MercadoBitcoin.com.br     32
BetsOfBitco.in            31
CoinURL.com               16
TheRockTrading.com        11
SatoshiDice.com            9
Bitfinex.com               8
BitNZ.com                  6
Cryptonit.net              6
Vircurex.com               6
OrderBook.net              5
MyBitcoin.com              5
FaucetBOX.com              4
McxNOW.com                 4
FYBSG.com                  3
Betcoins.net               2
BitMillions.com            2
Cryptsy.com                2
Just-Dice.com              2
BitAces.me                 1
Cryptorush.in              1
BTCGuild.com               1
Bitcoin-Roulette.com       1

In [8]:
def get_labels(cs_addr, known_addr_entity_dict):
    res = {}
    for idx, c in enumerate(cs_addr):
        for a in c:
            if a in known_addr:
                res[a] = idx
                
    labels_true = [] 
    labels_pred = []

    for a in res.keys():
        labels_true.append(known_addr_entity_dict[a])
        labels_pred.append(res[a])
    
    return labels_true, labels_pred

In [9]:
edge_tuples = df[["node1", "node2", "sum"]].itertuples(index=False)
g = Graph.TupleList(edge_tuples, directed=False, weights=True)
summary(g)

IGRAPH UNW- 139906 4587767 -- 
+ attr: name (v), weight (e)


In [10]:
def get_initial_and_fixed(sample_size):
    known_idxs = [idx for idx, addr in enumerate(g.vs["name"]) if addr in known_addr]

    initial = np.zeros(len(g.vs), dtype=np.int)
    
    rng = np.random.RandomState(32512163)
    
    sample_known_idxs = rng.choice(known_idxs, size=sample_size, replace=False) # TODO: set seed
    non_sample_idxs = np.setdiff1d(range(len(g.vs)), sample_known_idxs)

    for idx in sample_known_idxs:
        initial[idx] = known_addr_entity_dict[g.vs[idx]["name"]]

    initial[non_sample_idxs] = range(sample_size, len(g.vs))
    
    fixed = np.zeros(len(g.vs), dtype=np.int)
    fixed[sample_known_idxs] = 1
    fixed = fixed.astype(bool)
    
    return initial, fixed

In [11]:
for sample_size in list(range(0, len(known_addr), 2000)) + [len(known_addr)]:
    initial, fixed = get_initial_and_fixed(sample_size)
    cs = g.community_label_propagation(weights="weight", initial=initial, fixed=fixed)
    
    cs_addr = [g.vs.select(c)["name"] for c in cs]
    labels_true, labels_pred = get_labels(cs_addr=cs_addr, known_addr_entity_dict=known_addr_entity_dict)

    nmi = normalized_mutual_info_score(labels_true=labels_true, labels_pred=labels_pred)
    homog = homogeneity_score(labels_true=labels_true, labels_pred=labels_pred)
    
    print(f"{sample_size/len(g.vs):.2%} \t {sample_size/len(known_addr):.2%}   \t nmi: {nmi:.2f} \t homog: {homog:.2f}")


0.00% 	 0.00%   	 nmi: 0.50 	 homog: 0.68
1.43% 	 4.39%   	 nmi: 0.65 	 homog: 0.70
2.86% 	 8.77%   	 nmi: 0.67 	 homog: 0.71
4.29% 	 13.16%   	 nmi: 0.71 	 homog: 0.72
5.72% 	 17.54%   	 nmi: 0.71 	 homog: 0.73
7.15% 	 21.93%   	 nmi: 0.67 	 homog: 0.70
8.58% 	 26.31%   	 nmi: 0.70 	 homog: 0.73
10.01% 	 30.70%   	 nmi: 0.72 	 homog: 0.75
11.44% 	 35.09%   	 nmi: 0.64 	 homog: 0.60
12.87% 	 39.47%   	 nmi: 0.66 	 homog: 0.62
14.30% 	 43.86%   	 nmi: 0.76 	 homog: 0.79
15.72% 	 48.24%   	 nmi: 0.84 	 homog: 0.81
17.15% 	 52.63%   	 nmi: 0.85 	 homog: 0.82
18.58% 	 57.01%   	 nmi: 0.86 	 homog: 0.83
20.01% 	 61.40%   	 nmi: 0.87 	 homog: 0.85
21.44% 	 65.79%   	 nmi: 0.88 	 homog: 0.86
22.87% 	 70.17%   	 nmi: 0.89 	 homog: 0.88
24.30% 	 74.56%   	 nmi: 0.91 	 homog: 0.89
25.73% 	 78.94%   	 nmi: 0.92 	 homog: 0.91
27.16% 	 83.33%   	 nmi: 0.93 	 homog: 0.92
28.59% 	 87.71%   	 nmi: 0.94 	 homog: 0.93
30.02% 	 92.10%   	 nmi: 0.96 	 homog: 0.95
31.45% 	 96.48%   	 nmi: 0.98 	 homog: 0.9